Key Points:

Embedded Documents: Use embedded documents to store related data together, which allows for efficient querying.

The common attribute of these two dataset is date, if we do embedded data model, it would accelerate all the queries related with date, range of hotspot data would be limited to the range with this specific date.

In [ ]:
import os
print(os.getcwd())


In [ ]:
os.chdir("/home/student/A2/drive-download-20240507T073745Z-001")

In [ ]:
os.listdir()

In [ ]:
import pandas as pd
hotspot = pd.read_csv("./hotspot_historic.csv", sep=',')
climate = pd.read_csv("./climate_historic.csv", sep=',')

In [ ]:
hotspot.head()


In [ ]:
climate.head()

In [ ]:
climate.tail()

In [ ]:
print(climate['date'].dtype)

In [ ]:
import csv
from pprint import pprint

In [ ]:
def csv_to_dict(filename):
    with open(filename, mode='r', newline='', encoding='utf-8') as csvfile:
        #DictReader read the first row as column headers
        #return an iterator that produce disctionaries for each row
        #keys: column headers
        reader = csv.DictReader(csvfile)
        #convert iterator into list of dictionaries
        #keys: columne headers, values: cell values
        return list(reader)


# Read climate data
climate_data = csv_to_dict('climate_historic.csv')

# Read hotspot data
hotspot_data = csv_to_dict('hotspot_historic.csv')



In [ ]:
import pymongo
from pymongo import MongoClient
#Residence wifi ip address: 172.16.62.212
ipadd = "172.16.33.120"


client = MongoClient(ipadd, 27017)
db = client.fit3182_db
collection = db.fit3182_assignment_db

In [ ]:
#avoid error from exsiting collection with the same name
#collection.drop()

In [ ]:
# Convert the date columns to datetime objects for proper comparison
                        #convert from d/m/Y format into 
# climate['date'] = pd.to_datetime(climate['date'], format='%d/%m/%Y')
# hotspot['date'] = pd.to_datetime(hotspot['date'], format='%d/%m/%Y')

#since the locaiton of the sensors would not change across the dates
#since the station would not change location across the dates
# Embed fire data into climate data
# loop through each entry of the climate 
for index, climate_row in climate.iterrows():
    
    #filter out the climate data entries that have the sensor data
    #since they are important, others are not
    #since stations have different data according to dates

    #for each climate, filter out the hotspot/sensors with matching/corresponding date
    #list of true/false to filter out the hotspots related to the dates
    useful_dates = hotspot['date'] == climate_row['date']
    matching_fires = hotspot[useful_dates]

    #convert to list of disctionaries
    fires_list = matching_fires.to_dict('records')
    
    #inner-join
#     if fires_list:
        #print(fires_list)
    #left-outer join
    climate_doc = {
        'stations': climate_row['station'],
                                    #string from time with format, 
                                    #since MongoDB can not store object
        #.strftime('%d/%m/%Y')
        'date': climate_row['date'],
        'air_temperature_celcius': climate_row['air_temperature_celcius'],
        'relative_humidity': climate_row['relative_humidity'],
        'windspeed_knots': climate_row['windspeed_knots'],
        'max_wind_speed': climate_row['max_wind_speed'],
        'precipitation': climate_row['precipitation'],
        'GHI_w/m2': climate_row['GHI_w/m2'],
        #embed sensor data here
        'hotspots': fires_list
    }

    collection.insert_one(climate_doc)





In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
"""

# Q2.a.
res = collection.find(
    {
    'date': '12/12/2023'
    }
)

for d in res:
    print(d)

In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question b
"""

# as long as include filtering/matching on the documents of the hotspots array field in the main document
# use aggregate + pipeline

pipeline = [
    {   #first match, filter the station+date documents based on the hotspots that match
        '$match': {
            'hotspots.surface_temperature_celcius': {
                '$gte': 65,
                '$lte': 100
            }
        }
    },
    {   # since not all sensors in the document indexed by date and station
        # fulifll the condition, use unwind to release documents in array field(hotspots)
        # to flatten array
        '$unwind': '$hotspots'  # Unwind the hotspots array
    },
    {   #second match, filter the hotspot documents of the hotspots array
        '$match': {
            'hotspots.surface_temperature_celcius': {
                '$gte': 65,
                '$lte': 100
            }
        }
    },
    {
        '$project': { #   #value from output document after matching
                            #$ -> latitude field
            'latitude': '$hotspots.latitude',
            'longitude': '$hotspots.longitude',
            'surface_temperature_celcius': '$hotspots.surface_temperature_celcius',
            'confidence': '$hotspots.confidence'
        }
    }
]

res = collection.aggregate(pipeline)

# To display only the latitude
for d in res:
    pprint(d)


In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question c
"""
#filtering only on main document, no pipeline needed

res = collection.find(
    {
        'date':{
            '$in': ['15/12/2023', '16/12/2023']
        }
    },
    {   #only work for main 
        '_id': 0,
        'date': 1,
        'air_temperature_celcius': 1,
        'relative_humidity': 1,
                                        #must be used for array field
        'surface_temperature_celcius': '$hotspots.surface_temperature_celcius'
    }
)

for d in res:
    pprint(d)


In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question d
"""

pipeline = [
    {
        '$match': {
            'hotspots.confidence': {
                '$gte': 80,
                '$lte': 100
            }
        }
    },
    {
        '$unwind': '$hotspots'  # Corrected to a string
    },
    {
        '$match': {
            'hotspots.confidence': {
                '$gte': 80,
                '$lte': 100
            }
        }
    }
    # {   # no attributes from main document can be shown with fields of document in array field
    #     '$project': {
    #         'datetime': '$hotspots.datetime',
    #         'surface_temperature_celcius': '$hotspots.surface_temperature_celcius',
    #         'confidence': '$hotspots.confidence'  
    #     }
    # }
]

res = collection.aggregate(pipeline)

for d in res:
    pprint(d)


res = collection.aggregate(pipeline)

for d in res:
    pprint(d)

In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question e
"""
                        #sort 
#res = collection.find().sort('surface_temperature_celcius', -1).limit(10)
res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)

for i, d in enumerate(res):
    # only 10 records for res
    # if i == 9:
    pprint(d)

In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question f
"""

pipeline = [
    {
        '$group': {
            '_id': '$date',  # group by the 'date' field
            'count': {'$sum': 1}  # count the documents in each group
        }
    }
]

res = collection.aggregate(pipeline)
for i, d in enumerate(res):
    pprint(d)



In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question g
"""

#Fires records = fields from array field -> pipeline
pipeline = [
    {
        '$unwind': '$hotspots'  # Unwind the hotspots array
    },
    {
        '$match': {
            'hotspots.confidence': {
                '$lt': 70  # Match documents where confidence is less than 70
            }
        }
    },
    {
        '$project': {
            #Just Fire records
            '_id': 0,
            'confidence': '$hotspots.confidence',  
            'date': '$hotspots.date',  
            'datetime': '$hotspots.datetime',  
            'latitude': '$hotspots.latitude',  
            'longitude': '$hotspots.longitude', 
            'surface_temperature_celcius': '$hotspots.surface_temperature_celcius'
        }
    }
]

res = collection.aggregate(pipeline)
for i, d in enumerate(res):
    pprint(d)




In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question h
"""

pipeline = [
    {
        '$unwind': '$hotspots' 
    },
    {
        '$group': {
            #_id must exist in $group can be replaced
            '_id': '$hotspots.date',  # group by the date field from hotspots
            'avg_surface_temperature': {
                '$avg': '$hotspots.surface_temperature_celcius' 
            }
        }
    },
    {   #modify the project of the output after all above processing
        '$project': {
            '_id': 0,  # Exclude the _id field
            'date': '$_id',  # Map the _id field (which is the date) to 'date' in the output
            'avg_surface_temperature': 1  # 
        }
    },
    {
        '$sort': {'date': 1}  # Optional: sort by date
    }
]

res = collection.aggregate(pipeline)
for d in res:
    print(d)


In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question i
"""
# sort all by field 'GHI_w/m2' count for 10, -1 decending
res = collection.find().sort('GHI_w/m2', -1).limit(10)

for d in res:
    pprint(d)

In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question j
"""
#main document aggregate piepeline
pipeline = [
    {
        '$match':{
            'precipitation':{
                '$gte': 0.2,
                '$lte': 0.35
            }
        }
    },
    {
        '$project':{   
            '_id': 0,
            'date': 1,
            'precipitation': 1
        }
    }
]

res = collection.aggregate(pipeline)
for d in res:
    pprint(d)


In [ ]:
"""
Part A
Task 2. Querying MongoDBusingPyMongo
Question 3
"""

# Assuming 'db' is your MongoDB database object and 'climate_data' is your collection.
db.climate_data.create_index([
    ('date', pymongo.ASCENDING),
    ('station', pymongo.ASCENDING)
])


# since there is station for each day, with the compound index of date and station, we can efficiently access the data from climate, and the hotspots related to that date are limited, which can be further filtered with the locations of the hotspots by clutering.

In [ ]:
# # Query the collection using the compound index

# All documents for station 948700 on January 1, 2023
results = collection.find({
    'date': '31/12/2023',
    'station': 948700
})

# Process the results
for doc in results:
    pprint(doc)



In [ ]:
"""
Part b
Task 1. Processing Data Stream (45%)
Question a
"""
from datetime import datetime, timedelta

#res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)
res = collection.find().sort('date', pymongo.DESCENDING).limit(1)

for d in res:
    res = d
#     print(d)

latest_date = res['date']
#string to datetime object
# latest_date = datetime.strptime(latest_date, '%Y-%m-%d %H:%M:%S')
#string to datetime object
latest_date = datetime.strptime(latest_date, '%d/%m/%Y')
start_date = latest_date + timedelta(days=1)
print(start_date)

In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import json

hostip = "172.16.55.43" # change it to your IP

def connect_kafka_producer():
    producer = None
    
    try:
        producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],api_version=(0,10))
    
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
    finally:
        return producer 
    
def publish_msg(producer, topic, data):
    try:
        #print(data)        #send the  value by dumps to JSON object
                            #then encode with utf-8 into bytes
        value_bytes = json.dumps(data).encode('utf-8')
        producer.send(topic, value=value_bytes)
        #print(data)
        """
        The flush() method in Kafka’s producer API is used to ensure that all 
        previously sent messages have been transmitted to the server and acknowledged 
        before proceeding. When you call flush(), it blocks the current thread and waits 
        for the producer to complete the sending of all records
        """
        producer.flush()  
    except Exception as ex:
        print('Exception from publish_msg func')
        print(str(ex))
    


In [ ]:
"""
Event Producer 1
"""
climate_stream_csv = pd.read_csv("./climate_streaming.csv", sep=',')
climate_stream_csv.head()

In [ ]:
"""
Main Function
"""

if __name__ == '__main__':
    topic = 'Scenaries01'
    print('Publishing records..')
    producer01 = connect_kafka_producer()


    # Assuming 'climate' is your DataFrame
    #print(len(climate.index))
    rand_row_indices = list(range(len(climate.index)))
    random.shuffle(rand_row_indices)  
    #print(len(indices))

    for index, climate_row in climate_stream_csv.iterrows():
        climate_row = climate_stream_csv.loc[rand_row_indices[index]]

        #inner-join
    #     if fires_list:
            #print(fires_list)

        #left-outer join
        #print(start_date.strftime('%Y-%m-%d'))
        climate_doc = {
                                        #string from time with format, 
                                        #since MongoDB can not store object
            #.strftime('%d/%m/%Y')
            #convert datetime object to string in format
            # laster dumps not work on datetime object but number of string
            'latitude': float(climate_row['latitude']),
            'longtitude': float(climate_row['longitude']),
            'air_temperature_celcius': int(climate_row['air_temperature_celcius']),
            'relative_humidity': float(climate_row['relative_humidity']),
            'windspeed_knots': float(climate_row['windspeed_knots']),
            'max_wind_speed': float(climate_row['max_wind_speed']),
            #'precipitation ' blank space count as char in csv column
            # get to do try except, return None if column not exist
            #'precipitation ': str(climate_row.get(['precipitation '])),
            #'precipitation': str(climate_row.get(['precipitation'])),
            'precipitation': str(climate_row['precipitation']),
            'GHI_w/m2': int(climate_row['GHI_w/m2']),
            'producer': "Producer_1",
            'date': start_date.strftime('%Y-%m-%d')

        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        print(climate_doc)
        publish_msg(producer01, topic, climate_doc)
        start_date += timedelta(days=1)
        # change to 1 for debugging
        sleep(1)
        #sleep(10) # every 10 seconds send




In [ ]:
"""
Event Producer 2
"""
hotspot_AQUA = pd.read_csv("./hotspot_AQUA_streaming.csv", sep=',')

hotspot_AQUA.head()


"""
Main Function
"""
if __name__ == '__main__':
    topic = 'Scenaries02'
    print('Publishing records..')
    producer02 = connect_kafka_producer()
    
    
    rand_row_indices = list(range(len(hotspot_AQUA.index)))
    random.shuffle(rand_row_indices) 
    # random choose time from 0 to 10
    max_time = 10
    #rand_time = list(range(1,max_time))
#     random.shuffle(rand_time) 
     

    for index, hotspot_AQUA_row in hotspot_AQUA.iterrows():
        hotspot_AQUA_row = hotspot_AQUA.loc[rand_row_indices[index]]

        hotspot_AQUA_doc = {
            'latitude': hotspot_AQUA_row['latitude'],
            'longtitude': hotspot_AQUA_row['longitude'],
            'confidence': hotspot_AQUA_row['confidence'],
            'surface_temperature_celcius': hotspot_AQUA_row['surface_temperature_celcius'],
            'producer': "Producer_2",
            'random_time': rand_row_indices[index]%max_time + 1 # at least 1 sec
        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        #print(climate_doc)
        #print(hotspot_AQUA_doc['random_time'])
        publish_msg(producer02, topic, hotspot_AQUA_doc)
        # every random seconds
        sleep(hotspot_AQUA_doc['random_time']) 



In [ ]:
"""
Event Producer 3
"""
hotspot_TERRA= pd.read_csv("./hotspot_TERRA_streaming.csv", sep=',')

hotspot_TERRA.head()

"""
Main Function
"""
if __name__ == '__main__':
    topic = 'Scenaries03'
    print('Publishing records..')
    producer03 = connect_kafka_producer()
    
    
    rand_row_indices = list(range(len(hotspot_AQUA.index)))
    random.shuffle(rand_row_indices) 
    # random choose time from 0 to 10
    max_time = 10
    #rand_time = list(range(1,max_time))
#     random.shuffle(rand_time) 
     

    for index, hotspot_TERRA_row in hotspot_TERRA.iterrows():
        hotspot_TERRA_row = hotspot_TERRA.loc[rand_row_indices[index]]

        hotspot_TERRA_doc = {
            'latitude': hotspot_TERRA_row['latitude'],
            'longtitude': hotspot_TERRA_row['longitude'],
            'confidence': hotspot_TERRA_row['confidence'],
            'surface_temperature_celcius': hotspot_TERRA_row['surface_temperature_celcius'],
            'producer': "Producer_3",
            'random_time': rand_row_indices[index]%max_time + 1 # at least 1 sec
        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        #print(climate_doc)
        print(hotspot_TERRA_doc['random_time'])
        publish_msg(producer01, topic, hotspot_AQUA_doc)
        # every random seconds
        sleep(hotspot_TERRA_doc['random_time']) 

